In [1]:
# Install prerequisites

# Install PyCocoTools needed for FasterRCNN
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

# Copy useful functions from pytorch vision tools
%cp ../input/pytorch-vision-tools/references/detection/*.* .

  Cloning https://github.com/gautamchitnis/cocoapi.git (to revision cocodataset-master) to /tmp/pip-req-build-rjjn_xup
  Running command git clone -q https://github.com/gautamchitnis/cocoapi.git /tmp/pip-req-build-rjjn_xup


  Running command git checkout -b cocodataset-master --track origin/cocodataset-master


  Switched to a new branch 'cocodataset-master'
  Branch 'cocodataset-master' set up to track remote branch 'cocodataset-master' from 'origin'.


 \

 |

 /

 -

 \

 done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=272661 sha256=1da8f100c45fc833cfcfdd45cbdb5556f0ad44c071084851cb675e13cc128713
  Stored in directory: /tmp/pip-ephem-wheel-cache-l1q7ob13/wheels/6e/c9/59/56484d4d5ac1ab292a452b4c3870277256551505954fc4a1db
Successfully built pycocotools


In [2]:
# Import required libraries
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from os import path
import torch
import torchvision.utils
import utils
from engine import train_one_epoch, evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import Adam
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pickle
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Define a dataset class which defines how to load images,targets for training and validation
class VinBigDataset(Dataset):
    def __init__(self, img_dir, df, transforms, class_id=None):
        self.img_dir = img_dir
        if class_id:
            df = df[df["class_id"] == class_id]
        self.df = df
        self.imgs = df["image_id"].unique()
        self.transforms = transforms
        
    def __len__(self):
        # Return the number of elements in the dataset
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # The dataset iterates over each image id
        # Return the requested image,target from the dataset
        
        # Get the id of the current image
        img_id = self.imgs[idx]
        
        # Get the rows containing annotations for this image
        data_rows = self.df[self.df["image_id"] == img_id]
        boxes = data_rows[["x_min", "y_min", "x_max", "y_max"]].values
        
        # Convert into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # Load the image
        img_path = path.join(self.img_dir, f"{img_id}.png")
        img = Image.open(img_path).convert("RGB")
        
        # Compute the area of the annotated box
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        
        # Suppose all instances are not crowd (?)
        num_objs = len(boxes)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        labels = torch.tensor(data_rows["class_id"].values, dtype=torch.int64)
        
        # Define the target for this training data point
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img = self.transforms(img)
        
        return img, target

In [4]:
# Function to create transforms for preprocessing
def create_transforms(train):
    if train:
        return transforms.Compose([
            transforms.ToTensor(),
#             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    else:
        return transforms.Compose([
            transforms.ToTensor(),
#             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

In [5]:
# Function to create an instance of the model
def create_model():
    # The model has 15 classes (14 abnormalities and no abnormalities)
    num_classes = 15
    
    # Use resnet50 pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        
    # Fetch the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
    return model

## Training

In [6]:
# Load the training data csv file
data_dir = "../input/vinbig1024stratified/"

df_train = pd.read_csv(f"{data_dir}/train.csv")
df_train.fillna(0, inplace=True)
df_train.loc[df_train["class_id"] == 14, ['x_max', 'y_max']] = 1.0

# FasterRCNN handles class_id==0 as the background.
df_train["class_id"] = df_train["class_id"] + 1
df_train.loc[df_train["class_id"] == 15, ["class_id"]] = 0

# Print out the total number of images and the total number of annotations 
print(f"{df_train['image_id'].nunique()} images")
print(f"{len(df_train.index)} annotations")

11965 images
54046 annotations


In [7]:
# Create an instance of the dataset and transformations for training and validation

train_img_dir = '../input/vinbig1024stratified/output/dataset/train'
# train_img_dir = path.join(data_dir, 'train/train')
train_dataset = VinBigDataset(train_img_dir, df_train, create_transforms(train=True))

# Create data loaders for the training set. The collate function
# defines how to form a minibatch from the indiviaual data items. In our case we
# just want to collate them into a single list.

data_loader_train = DataLoader(\
    train_dataset, batch_size=5, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

# Print the number of elements in the test and training set
print(f"{len(train_dataset)} items in the training set")


11965 items in the training set


# Validation

In [8]:
# Load the VALIDATION data csv file
data_dir = "../input/vinbig1024stratified/"

df_val = pd.read_csv(f"{data_dir}/validation.csv")
df_val.fillna(0, inplace=True)
df_val.loc[df_val["class_id"] == 14, ['x_max', 'y_max']] = 1.0

# FasterRCNN handles class_id==0 as the background.
df_val["class_id"] = df_val["class_id"] + 1
df_val.loc[df_val["class_id"] == 15, ["class_id"]] = 0

# Print out the total number of images and the total number of annotations 
print(f"{df_val['image_id'].nunique()} images")
print(f"{len(df_val.index)} annotations")

1515 images
6867 annotations


In [9]:
# Create an instance of the dataset and transformations for training and validation

val_img_dir = '../input/vinbig1024stratified/output/dataset/validation/'
# train_img_dir = path.join(data_dir, 'train/train')
val_dataset = VinBigDataset(val_img_dir, df_val, create_transforms(train=False))

# Create data loaders for the training set. The collate function
# defines how to form a minibatch from the indiviaual data items. In our case we
# just want to collate them into a single list.

data_loader_val = DataLoader(\
    val_dataset, batch_size=5, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

# Print the number of elements in the test and training set
print(f"{len(val_dataset)} items in the validation set")

1515 items in the validation set


# Run the notebook

In [10]:
model = create_model()
use_pre_trained = True

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [11]:
# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

# Hyperparameters
learning_rate = 0.001
weight_decay = 0
num_epochs = 25

params = [p for p in model.parameters() if p.requires_grad]

optimizer = Adam(params, lr=learning_rate, weight_decay=weight_decay)
lr_scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.00002)

# Create a folder for saving the model weights
%mkdir model_weights
%mkdir stats

train_meters = []
coco_evals = []
t_start = time.time()

for epoch in range(num_epochs):
    # Train over the epoch
    logger = train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=100)
    train_meters.append(logger.meters)
    
    # Evaluate on the validation set
    coco_eval = evaluate(model, data_loader_val, device)
    coco_evals.append(coco_eval.coco_eval)
    
    # Update the learning rate
    lr_scheduler.step()

# Validation statistics for every class
coco_evals_class = []
for class_id in range(0,15):
    class_val_dataset = VinBigDataset(val_img_dir, df_val, create_transforms(train=False), class_id=class_id)

    data_loader_class_val = DataLoader(\
        class_val_dataset, batch_size=5, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)
    
    if len(class_val_dataset) == 0:
        continue
        
    coco_eval_class = evaluate(model, data_loader_class_val, device)
    coco_evals_class.append(coco_eval_class.coco_eval)
    
t_end = time.time()
print(f"Training took {t_end - t_start}")
      
# Save the model weights
torch.save(model.state_dict(), f"model_weights/model_weights.bin")

# Save the stats
pickle.dump(train_meters, open(f"stats/train_stats.pkl", 'wb'))
pickle.dump(coco_evals, open(f"stats/val_stats.pkl", 'wb'))
pickle.dump(coco_evals_class, open(f"stats/val_stats_2.pkl", 'wb'))
    

Epoch: [0]  [   0/2393]  eta: 2:04:33  lr: 0.000002  loss: 5.0222 (5.0222)  loss_classifier: 2.9089 (2.9089)  loss_box_reg: 0.0390 (0.0390)  loss_objectness: 1.5029 (1.5029)  loss_rpn_box_reg: 0.5714 (0.5714)  time: 3.1232  data: 1.3327  max mem: 3745


Epoch: [0]  [ 100/2393]  eta: 0:26:03  lr: 0.000102  loss: 0.4130 (1.0817)  loss_classifier: 0.2006 (0.5209)  loss_box_reg: 0.1208 (0.0765)  loss_objectness: 0.0547 (0.2775)  loss_rpn_box_reg: 0.0299 (0.2069)  time: 0.6501  data: 0.0344  max mem: 4170


Epoch: [0]  [ 200/2393]  eta: 0:24:31  lr: 0.000202  loss: 0.3095 (0.7159)  loss_classifier: 0.1315 (0.3315)  loss_box_reg: 0.0706 (0.0833)  loss_objectness: 0.0629 (0.1772)  loss_rpn_box_reg: 0.0216 (0.1239)  time: 0.6525  data: 0.0327  max mem: 4170


Epoch: [0]  [ 300/2393]  eta: 0:23:14  lr: 0.000302  loss: 0.4259 (0.6004)  loss_classifier: 0.1852 (0.2738)  loss_box_reg: 0.1312 (0.0902)  loss_objectness: 0.0647 (0.1419)  loss_rpn_box_reg: 0.0244 (0.0945)  time: 0.6536  data: 0.0330  max mem: 4171


Epoch: [0]  [ 400/2393]  eta: 0:22:05  lr: 0.000402  loss: 0.3740 (0.5401)  loss_classifier: 0.1142 (0.2403)  loss_box_reg: 0.1085 (0.0912)  loss_objectness: 0.0675 (0.1214)  loss_rpn_box_reg: 0.0739 (0.0871)  time: 0.6566  data: 0.0339  max mem: 4171


...

Epoch: [18]  [ 200/2393]  eta: 0:25:16  lr: 0.000198  loss: 0.5380 (0.6047)  loss_classifier: 0.0385 (0.0406)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.3584 (0.3756)  loss_rpn_box_reg: 0.1550 (0.1885)  time: 0.6928  data: 0.0361  max mem: 4171


Epoch: [18]  [ 300/2393]  eta: 0:23:59  lr: 0.000198  loss: 0.5349 (0.6003)  loss_classifier: 0.0364 (0.0392)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.3644 (0.3741)  loss_rpn_box_reg: 0.1366 (0.1870)  time: 0.6823  data: 0.0351  max mem: 4171


Epoch: [18]  [ 400/2393]  eta: 0:22:46  lr: 0.000198  loss: 0.5622 (0.5984)  loss_classifier: 0.0357 (0.0388)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.3658 (0.3737)  loss_rpn_box_reg: 0.1609 (0.1859)  time: 0.6807  data: 0.0339  max mem: 4171


Epoch: [18]  [ 500/2393]  eta: 0:21:35  lr: 0.000198  loss: 0.5732 (0.6039)  loss_classifier: 0.0272 (0.0397)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.3647 (0.3737)  loss_rpn_box_reg: 0.1752 (0.1904)  time: 0.6742  data: 0.0327  max mem: 4171


Epoch: [18]  [ 600/2393]  eta: 0:20:25  lr: 0.000198  loss: 0.6179 (0.6077)  loss_classifier: 0.0502 (0.0403)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.3746 (0.3739)  loss_rpn_box_reg: 0.1790 (0.1935)  time: 0.6741  data: 0.0333  max mem: 4171


In [ ]:
%%bash 
zip -r -q stats.zip stats/*.*

# Plot statistics

In [ ]:
# Functions for getting and plotting training stats

def get_train_stats(name, stats):
    return [s[name].value for s in stats]

def plot_train_stats(train_stats, name):
    loss = get_train_stats('loss', train_stats)
    loss_classifier = get_train_stats('loss_classifier', train_stats)
    loss_box_reg = get_train_stats('loss_box_reg', train_stats)
    loss_objectness = get_train_stats('loss_objectness', train_stats)
    loss_rpn_box_reg = get_train_stats('loss_rpn_box_reg', train_stats)
    # Plot the training stats
    fig = plt.figure(figsize=(12,8))
    plt.plot(loss, label='loss')
    plt.plot(loss_classifier, label='loss_classifier')
    plt.plot(loss_box_reg, label='loss_box_reg')
    plt.plot(loss_objectness, label='loss_objectness')
    plt.plot(loss_rpn_box_reg, label='loss_rpn_box_reg')
    plt.legend()
    plt.xticks(range(0, len(loss), 5))
    plt.xlabel('Epoch')
    plt.title(f'Training stats {name}')

In [ ]:
# Functions for getting and plotting validation stats

def get_val_stats(idx, stats):
    return [s['bbox'].stats[idx] for s in stats]

def plot_val_stats(val_stats, name):
    ap_50_95 = get_val_stats(0, val_stats)
    ap_50 = get_val_stats(1, val_stats)
    ap_75 = get_val_stats(2, val_stats)
    ar_all_md100 = get_val_stats(8, val_stats)

     # Plot the validation stats
    fig = plt.figure(figsize=(12,8))
    plt.plot(ap_50_95, label='AP .50:.95')
    plt.plot(ap_50, label='AP .50')
    plt.plot(ap_75, label='AP .75')
    plt.plot(ar_all_md100, label='AR All Max Dets 100')
    plt.legend()
    plt.xticks(range(0, len(ap_50), 5))
    plt.xlabel('Epoch')
    plt.title(f'Validation stats {name}')

In [ ]:
# Plot the stats
plot_train_stats(train_meters, 'baseline')
plot_val_stats(coco_evals, 'baseline')

In [ ]:
for i in range(15):
    plot_val_stats(coco_evals_class[i], f'baseline class {i}')
    

In [ ]:
coco_evals_class[0]['bbox']